In [97]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import json
import pickle
import pandas as pd
import progressbar
%matplotlib inline

In [6]:
np.random.seed(21)

In [8]:
from keras.models import load_model

In [20]:
df_cnts = pd.read_csv("../data/my_correct_counts.csv")

In [9]:
with open("../data/val.json", "r") as jfile:
    val_ids = json.load(jfile)
val_ids = [int(iid) for iid in val_ids]

In [10]:
pups_count_net = load_model("../data/models/class_pups/ckpt_cnt_04-3.38.h5")

In [15]:
def normalize_input(x_bgr):
    x = x_bgr.copy()
    x[..., 0] -= 103.939
    x[..., 1] -= 116.779
    x[..., 2] -= 123.68
    return x

In [28]:
np.sign(-1)

-1

In [77]:
def predict_pups_count(im_bgr, net, normalize_input, patch_size=(384, 384), offset=(0, 0), batch_size=8):
    # Add a black area around the image, the have the right number of sub-patches.
    # Image is centered + offset values.
    h, w, c = im_bgr.shape
    n_patches_y = h//patch_size[0]
    if h % patch_size[0] != 0:
        n_patches_y += 1
        
    n_patches_x = w//patch_size[1]
    if w % patch_size[0] != 0:
        n_patches_x += 1
        
    h_new = n_patches_y * patch_size[0]
    w_new = n_patches_x * patch_size[1]
    
    im_new = np.zeros((h_new, w_new, 3), dtype=np.uint8)
    dh = (h_new - h)//2
    dw = (w_new - w)//2

    ystart = dh + offset[0]
    if ystart < 0: ystart = 0
    yend = ystart + h
    if yend > h_new:
        yend = h_new
        ystart = yend - h
    
    xstart = dw + offset[1]
    if xstart < 0: xstart = 0
    xend = xstart + w
    if xend > w_new:
        xend = w_new
        xstart = xend - w
    
    im_new[ystart:yend, xstart:xend,...] = im_bgr
    
    # Predict the count on each patch
    # We fill a batch, and then predict for the full batch.
    counts = []
    batch_x = np.zeros((batch_size, ) + patch_size + (c, ), dtype=np.float32)
    
    current_batch_size = 0
    for py in range(n_patches_y):
        y_start = py * patch_size[0]
        y_end = y_start + patch_size[0]
        for px in range(n_patches_x):
            x_start = px * patch_size[1]
            x_end = x_start + patch_size[1]
            
            # Keep filling the batch
            batch_x[current_batch_size, :, :, :] = im_new[y_start:y_end, x_start:x_end, :]
            current_batch_size += 1
            
            if current_batch_size == batch_size or (py == n_patches_y - 1 and px == n_patches_x - 1) :
                # time to predict
                batch_x_normed = normalize_input(batch_x)
                batch_counts = net.predict(batch_x_normed)
                for c in batch_counts.ravel():
                    counts.append(c)
                current_batch_size = 0
    
    # The final count is just the sum over all patches.
    return np.sum(counts)

In [78]:
im = cv2.imread("/home/ubuntu/sealion/data/Train/10.jpg")

In [90]:
predict_pups_count(im, pups_count_net, normalize_input)

6.81 s ± 5.67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [91]:
def predict_all_pups_counts(image_dir, image_ids, net, normalize_input, offsets=[-16, 0, 16]):
    pups_counts = dict()
    with open(os.path.join(image_dir, "pups_mean.csv"), "w") as ofile: 
        for iid in image_ids:
            im = cv2.imread(os.path.join(image_dir, "{}.jpg".format(iid)))
            counts = []
            for offset_y in offsets:
                for offset_x in offsets:
                    count = predict_pups_count(im, net, normalize_input, offset=(offset_y, offset_x))
                    counts.append(count)
            mean_count = np.mean(counts)
            pups_counts[iid] = mean_count
            ofile.write("{},{}\n".format(iid, mean_count))
    return pups_counts

In [92]:
im_dir = "/home/ubuntu/sealion/data/Train/"
pups_counts = predict_all_pups_counts(im_dir, val_ids, pups_count_net, normalize_input)

In [105]:
def predict_all_pups_counts(image_dir, image_ids, net, normalize_input):
    pups_counts = dict()
    with open(os.path.join(image_dir, "pups_mean.csv"), "w") as ofile: 
        with progressbar.ProgressBar(max_value=len(image_ids)) as bar:     
            for i, iid in enumerate(image_ids):
                im = cv2.imread(os.path.join(image_dir, "{}.jpg".format(iid)))
                count = predict_pups_count(im, net, normalize_input)
                ofile.write("{},{}\n".format(iid, count))
                bar.update(i)
    return pups_counts

In [106]:
with open("../data/test_sub.json", "r") as jfile:
    test_ids = json.load(jfile)
test_ids = [int(iid) for iid in test_ids]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [ ]:
im_dir = "/home/ubuntu/data/sealion/data/Test/"
pups_counts = predict_all_pups_counts(im_dir, test_ids, pups_count_net, normalize_input)

100% (18636 of 18636) |#####| Elapsed Time: 1 day, 7:11:45 Time: 1 day, 7:11:45


In [ ]:
1+1